## 初期値

In [1]:
import os

os.chdir("/work/app")

# %% 準備
import pandas as pd
from utils.logger import app_logger
from utils.config_loader import get_template_config
from logic.manage.utils.csv_loader import load_all_filtered_dataframes
from logic.controllers.csv_controller import apply_expected_dtypes
from logic.manage.utils.load_template import load_master_and_template
from IPython.display import display
import re
from logic.manage.factory_report import process
from logic.manage.utils.excel_tools import create_label_rows_generic, sort_by_cell_row
from utils.value_setter import set_value_fast,set_value_fast_safe
from logic.manage.utils.summary_tools import summary_apply


# 表示ラベルマップ（処理対象名として使う）
csv_label_map = {"yard": "ヤード一覧", "shipping": "出荷一覧", "receive": "受入一覧"}

debug_receive = "/work/app/data/output/debug_receive.parquet"
debug_shipping = "/work/app/data/output/debug_shipping.parquet"
debug_yard = "/work/app/data/output/debug_yard.parquet"

dfs = {
    "receive": pd.read_parquet(debug_receive),
    "shipping": pd.read_parquet(debug_shipping),
    "yard": pd.read_parquet(debug_yard),
}  # テスト用CSV
# dfs
df_shipping = dfs["shipping"]
df_yard = dfs["yard"]
df_receive = dfs["receive"]

2025-05-08 13:16:32,081 [INFO] (__init__.py:24) [dd3b4088c738/root] ✅ factory_report.py の process 関数を登録しました
2025-05-08 13:16:32,086 [INFO] (__init__.py:24) [dd3b4088c738/root] ✅ balance_sheet.py の process 関数を登録しました
2025-05-08 13:16:32,095 [INFO] (__init__.py:24) [dd3b4088c738/root] ✅ average_sheet.py の process 関数を登録しました
2025-05-08 13:16:32,100 [INFO] (__init__.py:24) [dd3b4088c738/root] ✅ management_sheet.py の process 関数を登録しました
2025-05-08 04:16:32.323 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
df_yard

,伝票日付,取引先名,品名,正味重量,数量,単位名,単価,金額,営業担当者名,業者CD,業者名,種類CD,種類名,品名CD
0,2025-03-31,ｵﾈｽﾄ･ﾜﾝ,選別,80,80,kg,60.0,"4,800",オネスト・ワン,1308,ｵﾈｽﾄ･ﾜﾝ,1,処分費,30
1,2025-03-31,泉土木,GD,110,110,kg,14.0,"1,540",矢作,3140,ｻﾞｲﾏｯｸｽ,3,有価物,69
2,2025-03-31,泉土木,GC 軽鉄･ｽﾁｰﾙ類,810,810,kg,24.5,"19,845",矢作,9026,ﾚｱﾘﾅ,3,有価物,68
3,2025-03-31,泉宣宏社,GC 軽鉄･ｽﾁｰﾙ類,310,310,kg,0.0,0,渡辺,311,泉宣宏社,3,有価物,68
4,2025-03-31,港区教育委員会事務局学校教育部,混合廃棄物,480,480,kg,45.0,"21,600",小林,923,港区教育委員会事務局学校教育部,1,処分費,14
5,2025-03-31,港区教育委員会事務局学校教育部,混合廃棄物A,130,130,kg,65.0,"8,450",小林,923,港区教育委員会事務局学校教育部,1,処分費,1
6,2025-03-31,五十嵐商会,選別,460,460,kg,65.0,"29,900",渡辺,7996,ｼﾞｪｲｼﾃｨｰ,1,処分費,30
7,2025-03-31,ﾋﾗｶﾜ,GC 軽鉄･ｽﾁｰﾙ類,1470,"1,470",kg,0.0,0,川部,428,ﾋﾗｶﾜ,3,有価物,68
8,2025-03-31,ﾅｲﾃｯｸｽ,選別,1220,"1,220",kg,70.0,"85,400",山本,6509,ﾅｲﾃｯｸｽ,1,処分費,30
9,2025-03-31,ﾃﾙｳｪﾙ東日本,選別(ｷｬﾋﾞﾈｯﾄ),650,650,kg,65.0,"42,250",渡辺,4418,ﾃﾙｳｪﾙ東日本,1,処分費,171


In [3]:
df_shipping

,伝票日付,出荷番号,取引先名,業者CD,業者名,現場CD,現場名,品名,正味重量,数量,単位名,単価,金額,運搬業者名,伝票区分名
0,2025-03-31,10499,■I ･ S ｱｲ･ｴｽ,8327,丸源起業,0,丸源起業ひかりｴｺｽﾃｰｼｮﾝ,混合廃棄物(A),13870,"13,870",kg,22,"305,140",ｵﾈｽﾄ,支払
1,2025-03-31,10542,東京ﾎﾞｰﾄﾞ工業,5238,東京ﾎﾞｰﾄﾞ工業,0,東京ﾎﾞｰﾄﾞ工場新木場ﾘｻｲｸﾘﾝｸﾞ工場,木くず,2620,"2,620",kg,7,"18,340",東亜物流,支払
2,2025-03-31,10543,ｵﾈｽﾄ,1,ｵﾈｽﾄ,2,ｵﾈｽﾄ新木場ﾔｰﾄﾞ,金属くず ｼｭﾚｯﾀ,730,730,kg,0,0,■ｴｺﾗｲﾝ,売上
3,2025-03-31,10545,ｵﾈｽﾄ,1,ｵﾈｽﾄ,2,ｵﾈｽﾄ新木場ﾔｰﾄﾞ,鉄千地,1460,"1,460",kg,0,0,東亜物流,支払
4,2025-03-31,10546,宮崎,747,宮崎,1,宮崎新木場リサイクルセンター,ﾀﾞﾝﾎﾞｰﾙ,250,250,kg,14,"3,500",■ｴｺﾗｲﾝ,売上
5,2025-03-31,10547,東京ﾎﾞｰﾄﾞ工業,5238,東京ﾎﾞｰﾄﾞ工業,0,東京ﾎﾞｰﾄﾞ工場新木場ﾘｻｲｸﾘﾝｸﾞ工場,木くず,2050,"2,050",kg,7,"14,350",■ｴｺﾗｲﾝ,支払
6,2025-03-31,10548,ADVANCE,8327,丸源起業,0,丸源起業ひかりｴｺｽﾃｰｼｮﾝ,混合廃棄物(A),0,"13,850",kg,1,"13,850",ｵﾈｽﾄ,支払
7,2025-03-31,10541,東京産資源,8425,東京産資源,0,東京産資源,GB(ｷﾞﾛA),500,500,kg,33.5,"16,750",ｵﾈｽﾄ,売上
8,2025-03-31,10549,ADVANCE,8327,丸源起業,0,丸源起業ひかりｴｺｽﾃｰｼｮﾝ,混合廃棄物(B),0,"10,590",kg,1,"10,590",ｵﾈｽﾄ,支払
9,2025-03-31,10551,東京産資源,8425,東京産資源,1,【ﾔｰﾄﾞ】,GB(ｷﾞﾛA),3810,"3,810",kg,31.5,"120,015",東京産資源,売上


In [4]:
df_receive

,伝票日付,売上日付,支払日付,業者CD,業者名,伝票区分CD,伝票区分名,品名CD,品名,正味重量,数量,単位CD,単位名,単価,金額,受入番号,集計項目CD,集計項目
0,2025-03-31,2025/03/31(月),2025/03/31(月),10,ｸﾛｻﾜ工業,1,売上,2,混合廃棄物B,640,640,3,kg,70,44800.0,46663,3,C（解体）
1,2025-03-31,2025/03/31(月),2025/03/31(月),5515,ｷｬﾌﾟﾃｨｿﾘｭｰｼｮﾝｽﾞ,1,売上,350,運搬費(4ｔﾕﾆｯｸ車),0,1,7,台,"30,000",30000.0,46660,4,D（建設）
2,2025-03-31,2025/03/31(月),2025/03/31(月),5515,ｷｬﾌﾟﾃｨｿﾘｭｰｼｮﾝｽﾞ,2,支払,68,GC 軽鉄･ｽﾁｰﾙ類,480,480,3,kg,15.5,7440.0,46660,4,D（建設）
3,2025-03-31,2025/03/31(月),2025/03/31(月),5676,千代田電興,1,売上,1,混合廃棄物A,300,300,3,kg,65,19500.0,46674,4,D（建設）
4,2025-03-31,2025/03/31(月),2025/03/31(月),5854,沖原工務所,1,売上,2,混合廃棄物B,70,70,3,kg,70,4900.0,46743,3,C（解体）
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,2025-03-31,2025/03/31(月),2025/03/31(月),1656,真和商事,1,売上,93,混合廃棄物(ﾋﾞﾝ･缶･ﾍﾟｯﾄﾎﾞﾄﾙ),120,120,3,kg,45,5400.0,46625,2,B（残置）
191,2025-03-31,2025/03/31(月),2025/03/31(月),1656,真和商事,1,売上,93,混合廃棄物(ﾋﾞﾝ･缶･ﾍﾟｯﾄﾎﾞﾄﾙ),140,140,3,kg,45,6300.0,46634,2,B（残置）
192,2025-03-31,2025/03/31(月),2025/03/31(月),2246,東ｿｰ物流,1,売上,1,混合廃棄物A,730,730,3,kg,60,43800.0,46635,2,B（残置）
193,2025-03-31,2025/03/31(月),2025/03/31(月),955,CRASHIP,1,売上,2,混合廃棄物B,380,380,3,kg,70,26600.0,46710,3,C（解体）


### Process

In [5]:
# 処理の統合
def process1(dfs: dict) -> pd.DataFrame:
    logger = app_logger()
    """
    Streamlitの選択に基づき、工場日報（処分パート）を処理するエントリーポイント関数。
    """

    logger = app_logger()

    # --- テンプレート設定の取得 ---
    template_key = "balance_sheet"
    template_config = get_template_config()[template_key]
    template_name = template_config["key"]
    csv_keys = template_config["required_files"]
    logger.info(f"[テンプレート設定読込] key={template_key}, files={csv_keys}")

    # --- CSVの調整・読み込み ---
    dfs = apply_expected_dtypes(dfs, template_key)
    df_dict = load_all_filtered_dataframes(dfs, csv_keys, template_name)
    df_receive = df_dict.get("receive")
    df_shipping = df_dict.get("shipping")
    df_yard = df_dict.get("yard")

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["factory"]
    master_csv = load_master_and_template(master_path)


    return df_receive, df_yard, df_shipping, master_csv


df_receive, df_yard, df_shipping ,master_csv= process1(dfs)

2025-05-08 13:16:32,543 [INFO] (1895370545.py:15) [dd3b4088c738/root] [テンプレート設定読込] key=balance_sheet, files=['receive', 'yard', 'shipping']
2025-05-08 13:16:32,576 [INFO] (csv_controller.py:42) [dd3b4088c738/root] ✅ 型を適用しました: receive
2025-05-08 13:16:32,591 [INFO] (csv_controller.py:42) [dd3b4088c738/root] ✅ 型を適用しました: shipping
2025-05-08 13:16:32,602 [INFO] (csv_controller.py:42) [dd3b4088c738/root] ✅ 型を適用しました: yard


In [6]:
display(df_receive.head())
display(df_yard.head())
display(df_shipping.head())
display(master_csv)

,受入番号,正味重量,伝票区分名,単位名,単価,金額,品名,数量
0,46663,640,売上,kg,70,44800,混合廃棄物B,640
1,46660,0,売上,台,30000,30000,運搬費(4ｔﾕﾆｯｸ車),1
2,46660,480,支払,kg,15,7440,GC 軽鉄･ｽﾁｰﾙ類,480
3,46674,300,売上,kg,65,19500,混合廃棄物A,300
4,46743,70,売上,kg,70,4900,混合廃棄物B,70


,種類名,品名,正味重量
0,処分費,選別,80
1,有価物,GD,110
2,有価物,GC 軽鉄･ｽﾁｰﾙ類,810
3,有価物,GC 軽鉄･ｽﾁｰﾙ類,310
4,処分費,混合廃棄物,480


,業者CD,業者名,金額,品名,正味重量
0,8327,丸源起業,305140,混合廃棄物(A),13870
1,5238,東京ﾎﾞｰﾄﾞ工業,18340,木くず,2620
2,1,ｵﾈｽﾄ,0,金属くず ｼｭﾚｯﾀ,730
3,1,ｵﾈｽﾄ,0,鉄千地,1460
4,747,宮崎,3500,ﾀﾞﾝﾎﾞｰﾙ,250


,大項目,セル,値
0,搬出量,E6,NaN
1,処分費,E7,NaN
2,有価物,E8,NaN


In [7]:
def process2(df_receive, df_yard, df_shipping):
    # --- 個別処理 ---
    logger.info("▶️ 搬出量データ処理開始")
    master_csv_facotry = process_factory_report(dfs)

    logger.info("▶️ 処分費データ処理開始")
    master_csv_facotry = process_syobun_cost(df_yard, df_shipping)
    display(master_csv_facotry)

    return

## 排出量（工場日報から）

In [6]:

def reflect_total_from_factory(master_csv, df_factory):
    from utils.value_setter import set_value_fast,set_value_fast_safe
    total_sum = df_factory.loc[df_factory["大項目"] == "総合計", "値"].squeeze()

    match_columns = ["大項目"]
    match_value = ["搬出量"]
    master_csv=set_value_fast_safe(master_csv, match_columns, match_value, total_sum)

    return master_csv

In [7]:

def process_factory_report(dfs,master_csv):
    logger = app_logger()
    from logic.manage.factory_report import process
    from utils.value_setter import set_value_fast,set_value_fast_safe


    # 工場日報からdfを読込
    df_factory = process(dfs)

    # 搬出量を抜出
    after_master_csv = reflect_total_from_factory(master_csv, df_factory)

    return after_master_csv


master_csv1 = process_factory_report(dfs,master_csv)
master_csv1

2025-05-02 15:42:31,079 [INFO] (factory_report.py:27) [dd3b4088c738/root] [テンプレート設定読込] key=factory_report, files=['yard', 'shipping']
2025-05-02 15:42:31,101 [INFO] (factory_report.py:35) [dd3b4088c738/root] ▶️ 出荷処分データ処理開始
/work/app/utils/data_cleaning.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['8339' '5238' '8340' '8085' '6421' '8337' '8215' '8336' '8327' '8327'
 '8327' '8327' '705' '1185' '8076' '8077' '5199' '8334' '6797' '8338'
 '8341' '8080' '349' '6786' '5227' '8329' '8360' '8363' '8398' '8679'
 '339' '8082']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[valid, col] = df.loc[valid, col].apply(lambda x: str(int(float(x))))
2025-05-02 15:42:31,147 [INFO] (factory_report_shobun.py:40) [dd3b4088c738/root] ✅ 出荷処分の帳票生成が完了しました。
2025-05-02 15:42:31,148 [INFO] (factory_report.py:38) [dd3b4088c738/root] ▶️ 出荷有価データ処理開始
2025-05-02 15:42:31,176 [INFO]

2025-05-02 15:42:31,247 [INFO] (factory_report_yuuka.py:56) [dd3b4088c738/root] ✅ 出荷有価の帳票生成が完了しました。
2025-05-02 15:42:31,249 [INFO] (factory_report.py:41) [dd3b4088c738/root] ▶️ 出荷ヤードデータ処理開始
2025-05-02 15:42:31,280 [INFO] (summary_tools.py:132) [dd3b4088c738/root] ▶️ 処理対象シート: ヤード, キー: ['種類名'], 集計列: 正味重量
2025-05-02 15:42:31,297 [INFO] (summary_tools.py:132) [dd3b4088c738/root] ▶️ 処理対象シート: ヤード, キー: ['種類名', '品名'], 集計列: 正味重量
2025-05-02 15:42:31,317 [INFO] (summary_tools.py:132) [dd3b4088c738/root] ▶️ 処理対象シート: 出荷, キー: ['業者名', '品名'], 集計列: 正味重量
2025-05-02 15:42:31,336 [INFO] (factory_report_yard.py:44) [dd3b4088c738/root] ✅ 出荷ヤードの帳票生成が完了しました。
2025-05-02 15:42:31,337 [INFO] (factory_report.py:45) [dd3b4088c738/root] 🧩 各処理結果を結合中...
2025-05-02 15:42:31,371 [DEBUG] (factory_report.py:59) [dd3b4088c738/root] 
              大項目   セル             値 カテゴリ
0              月日   O1         3月31日  NaN
1              和暦   M1          令和7年  NaN
2                  F19       紙、ダンボール   有価
3                  I19  

,大項目,セル,値
0,搬出量,E6,101130.0
1,処分費,E7,NaN


## 仕入処分費

In [8]:
def process_syobun_cost(df_yard, df_shipping):

    logger = app_logger()

    # 出荷一覧処理
    # 処分費
    master_csv_shobun_cost = shobun_cost(df_shipping)

    # 金庫

    # ヤード一覧処理

    # 金庫_処分費

    return

In [9]:
master_csv1

,大項目,セル,値
0,搬出量,E6,101130.0
1,処分費,E7,NaN


In [10]:
df_shipping

,業者CD,業者名,金額,品名,正味重量
0,8327,丸源起業,305140,混合廃棄物(A),13870
1,5238,東京ﾎﾞｰﾄﾞ工業,18340,木くず,2620
2,1,ｵﾈｽﾄ,0,金属くず ｼｭﾚｯﾀ,730
3,1,ｵﾈｽﾄ,0,鉄千地,1460
4,747,宮崎,3500,ﾀﾞﾝﾎﾞｰﾙ,250
5,5238,東京ﾎﾞｰﾄﾞ工業,14350,木くず,2050
6,8327,丸源起業,13850,混合廃棄物(A),0
7,8425,東京産資源,16750,GB(ｷﾞﾛA),500
8,8327,丸源起業,10590,混合廃棄物(B),0
9,8425,東京産資源,120015,GB(ｷﾞﾛA),3810


### 処分費

In [ ]:
def shobun_cost(df_shipping):

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["shobun_cost"]
    master_csv = load_master_and_template(master_path)

    # # 👇 業者CDを文字列に揃える
    df_shipping["業者CD"] = df_shipping["業者CD"].astype(str)
    master_csv["業者CD"] = master_csv["業者CD"].astype(str)

    # 処分費の総計
    key_cols = ["業者CD"]
    source_col = "金額"
    master_csv = summary_apply(
        master_csv,
        df_shipping,
        key_cols,
        source_col,
    )
    return master_csv


master_csv_shobun_cost = shobun_cost(df_shipping)
display(master_csv_shobun_cost)

/tmp/ipykernel_273019/3117396381.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shipping["業者CD"] = df_shipping["業者CD"].astype(str)
2025-05-02 15:42:31,461 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['業者CD'], 集計列=金額 ➡ 書き込み列=値


,業者CD,業者名,セル,値
0,8339,共同土木,NaN,97300.0
1,5238,東京ボード,NaN,72940.0
2,8340,木材開発,NaN,26800.0
3,8085,松岡環境開発,NaN,NaN
4,6421,アクトリー,NaN,NaN
5,8337,丸幸,NaN,NaN
6,8215,杉田建材,NaN,424960.0
7,8336,新潟環境,NaN,NaN
8,8327,丸源企業（ S ）,NaN,636980.0
9,705,北陸環境サービス,NaN,NaN


### 処分費_金庫

In [12]:
from utils.config_loader import get_unit_price_table_csv
from logic.manage.utils.multiply_tools import multiply_columns


def shobun_cost_kinko(df_shipping: pd.DataFrame) -> pd.DataFrame:
    """
    出荷データから処分コストを計算する関数

    Parameters
    ----------
    df_shipping : pd.DataFrame
        出荷データのDataFrame

    Returns
    -------
    pd.DataFrame
        単価×正味重量の「値」を含むマスターDataFrame
    """

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["syobun_cost_kinko"]
    master_csv = load_master_and_template(master_path)

    # --- ② 正味重量の更新 ---
    key_cols = ["業者名", "品名"]
    source_col = "正味重量"
    master_with_weight = summary_apply(
        master_csv,
        df_shipping,
        key_cols,
        source_col,
        source_col,  # target_colとsource_colが同じ
    )

    # --- ③ 単価の設定 ---
    unit_price_csv = get_unit_price_table_csv()
    master_with_price = summary_apply(
        master_with_weight,
        unit_price_csv,
        key_cols,
        source_col="設定単価",
        target_col="設定単価",
    )

    # --- ④ 単価×正味重量の計算 ---
    result_df = multiply_columns(
        master_with_price, col1="設定単価", col2="正味重量", result_col="値"
    )

    return result_df


# --- 呼び出し例 ---
master_csv_shobun_cost = shobun_cost_kinko(df_shipping)
display(master_csv_shobun_cost)

2025-05-02 15:42:31,515 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['業者名', '品名'], 集計列=正味重量 ➡ 書き込み列=正味重量
2025-05-02 15:42:31,538 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['業者名', '品名'], 集計列=設定単価 ➡ 書き込み列=設定単価


,大項目,業者名,品名,セル,値,正味重量,設定単価
0,金庫,ｵﾈｽﾄ,金庫,NaN,4800.0,80.0,60.0


### 処分費_金庫_ヤード

In [13]:
from utils.config_loader import get_unit_price_table_csv
from logic.manage.utils.multiply_tools import multiply_columns


def shobun_cost_yard(df_yard: pd.DataFrame) -> pd.DataFrame:
    """
    Yardデータ（ヤード集荷）から処分コストを計算する関数。

    マスターCSVとYard出荷データをマージし、
    正味重量および単価を適用した上で、
    最終的な「単価 × 正味重量」の金額列（値）を計算する。

    Parameters
    ----------
    df_yard : pd.DataFrame
        Yard出荷データのDataFrame。

    Returns
    -------
    pd.DataFrame
        単価・正味重量・および「単価 × 正味重量（値）」を含む更新後のマスターDataFrame。
    """

    # --- ① マスターCSVの読み込み ---
    config = get_template_config()["balance_sheet"]
    master_path = config["master_csv_path"]["syobun_cost_kinko_yard"]
    master_csv = load_master_and_template(master_path)

    # --- ② 正味重量の更新 ---
    key_cols = ["種類名", "品名"]
    source_col = "正味重量"
    master_with_weight = summary_apply(
        master_csv,
        df_yard,
        key_cols,
        source_col,
        source_col,  # target_colとsource_colが同じ
    )

    # --- ③ 単価の設定 ---
    unit_price_csv = get_unit_price_table_csv()
    master_with_price = summary_apply(
        master_with_weight,
        unit_price_csv,
        key_cols,
        source_col="設定単価",
        target_col="設定単価",
    )

    # --- ④ 単価×正味重量の計算 ---
    result_df = multiply_columns(
        master_with_price, col1="設定単価", col2="正味重量", result_col="値"
    )

    return result_df


# --- 呼び出し例 ---
master_csv_shobun_cost = shobun_cost_yard(df_yard)
display(master_csv_shobun_cost)

2025-05-02 15:42:31,581 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['種類名', '品名'], 集計列=正味重量 ➡ 書き込み列=正味重量
2025-05-02 15:42:31,603 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['種類名', '品名'], 集計列=設定単価 ➡ 書き込み列=設定単価


,大項目,種類名,品名,セル,値,正味重量,設定単価
0,金庫,処分費,金庫,NaN,10800.0,180.0,60.0


### 処分費合計

In [14]:
master_csv1

,大項目,セル,値
0,搬出量,E6,101130.0
1,処分費,E7,NaN


In [ ]:
def sum_shobun(df_shobun,df_kinko, df_kinko_yard):

    # 列名を変更
    df_shobun = df_shobun.rename(columns={"業者CD": "大項目"})
    sum_shobun_total = df_shobun["値"].sum()

    # 合計
    match_columns = ["大項目"]
    match_value = ["処分費"]
    master_csv2 = set_value_fast_safe(master_csv1, match_columns, match_value, sum_shobun_total)

    return master_csv2

df_shobun = shobun_cost(df_shipping)
df_kinko = shobun_cost_kinko(df_shipping)
df_kinko_yard = shobun_cost_yard(df_yard)

master_csv2 = sum_shobun(df_shobun, df_kinko, df_kinko_yard)
master_csv2

/tmp/ipykernel_273019/3117396381.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shipping["業者CD"] = df_shipping["業者CD"].astype(str)
2025-05-02 15:42:31,661 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['業者CD'], 集計列=金額 ➡ 書き込み列=値
2025-05-02 15:42:31,730 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['業者名', '品名'], 集計列=正味重量 ➡ 書き込み列=正味重量
2025-05-02 15:42:31,754 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['業者名', '品名'], 集計列=設定単価 ➡ 書き込み列=設定単価
2025-05-02 15:42:31,785 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['種類名', '品名'], 集計列=正味重量 ➡ 書き込み列=正味重量
2025-05-02 15:42:31,812 [INFO] (summary_tools.py:78) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['種類名', '品名'], 集計列=設定単価 ➡ 書き込み列=設

,大項目,セル,値
0,搬出量,E6,101130.0
1,処分費,E7,1673600.0


## 有価物

In [34]:
display(master_csv)
display(df_shipping)
display(df_yard)

,業者名,値
0,ｴｺﾏｯｸｽ,NaN
1,宮崎,NaN
2,Y-maxx,NaN
3,ﾐﾀﾞｯｸ,NaN
4,東京紙業城東事業所,NaN
5,新洋,NaN
6,荒井商店【二次】,NaN


,伝票日付,出荷番号,取引先名,業者CD,業者名,現場CD,現場名,品名,正味重量,数量,単位名,単価,金額,運搬業者名,伝票区分名
0,2025-03-31,10499,■I ･ S ｱｲ･ｴｽ,8327,丸源起業,0,丸源起業ひかりｴｺｽﾃｰｼｮﾝ,混合廃棄物(A),13870,"13,870",kg,22,"305,140",ｵﾈｽﾄ,支払
1,2025-03-31,10542,東京ﾎﾞｰﾄﾞ工業,5238,東京ﾎﾞｰﾄﾞ工業,0,東京ﾎﾞｰﾄﾞ工場新木場ﾘｻｲｸﾘﾝｸﾞ工場,木くず,2620,"2,620",kg,7,"18,340",東亜物流,支払
2,2025-03-31,10543,ｵﾈｽﾄ,1,ｵﾈｽﾄ,2,ｵﾈｽﾄ新木場ﾔｰﾄﾞ,金属くず ｼｭﾚｯﾀ,730,730,kg,0,0,■ｴｺﾗｲﾝ,売上
3,2025-03-31,10545,ｵﾈｽﾄ,1,ｵﾈｽﾄ,2,ｵﾈｽﾄ新木場ﾔｰﾄﾞ,鉄千地,1460,"1,460",kg,0,0,東亜物流,支払
4,2025-03-31,10546,宮崎,747,宮崎,1,宮崎新木場リサイクルセンター,ﾀﾞﾝﾎﾞｰﾙ,250,250,kg,14,"3,500",■ｴｺﾗｲﾝ,売上
5,2025-03-31,10547,東京ﾎﾞｰﾄﾞ工業,5238,東京ﾎﾞｰﾄﾞ工業,0,東京ﾎﾞｰﾄﾞ工場新木場ﾘｻｲｸﾘﾝｸﾞ工場,木くず,2050,"2,050",kg,7,"14,350",■ｴｺﾗｲﾝ,支払
6,2025-03-31,10548,ADVANCE,8327,丸源起業,0,丸源起業ひかりｴｺｽﾃｰｼｮﾝ,混合廃棄物(A),0,"13,850",kg,1,"13,850",ｵﾈｽﾄ,支払
7,2025-03-31,10541,東京産資源,8425,東京産資源,0,東京産資源,GB(ｷﾞﾛA),500,500,kg,33.5,"16,750",ｵﾈｽﾄ,売上
8,2025-03-31,10549,ADVANCE,8327,丸源起業,0,丸源起業ひかりｴｺｽﾃｰｼｮﾝ,混合廃棄物(B),0,"10,590",kg,1,"10,590",ｵﾈｽﾄ,支払
9,2025-03-31,10551,東京産資源,8425,東京産資源,1,【ﾔｰﾄﾞ】,GB(ｷﾞﾛA),3810,"3,810",kg,31.5,"120,015",東京産資源,売上


,伝票日付,取引先名,品名,正味重量,数量,単位名,単価,金額,営業担当者名,業者CD,業者名,種類CD,種類名,品名CD
0,2025-03-31,ｵﾈｽﾄ･ﾜﾝ,選別,80,80,kg,60.0,"4,800",オネスト・ワン,1308,ｵﾈｽﾄ･ﾜﾝ,1,処分費,30
1,2025-03-31,泉土木,GD,110,110,kg,14.0,"1,540",矢作,3140,ｻﾞｲﾏｯｸｽ,3,有価物,69
2,2025-03-31,泉土木,GC 軽鉄･ｽﾁｰﾙ類,810,810,kg,24.5,"19,845",矢作,9026,ﾚｱﾘﾅ,3,有価物,68
3,2025-03-31,泉宣宏社,GC 軽鉄･ｽﾁｰﾙ類,310,310,kg,0.0,0,渡辺,311,泉宣宏社,3,有価物,68
4,2025-03-31,港区教育委員会事務局学校教育部,混合廃棄物,480,480,kg,45.0,"21,600",小林,923,港区教育委員会事務局学校教育部,1,処分費,14
5,2025-03-31,港区教育委員会事務局学校教育部,混合廃棄物A,130,130,kg,65.0,"8,450",小林,923,港区教育委員会事務局学校教育部,1,処分費,1
6,2025-03-31,五十嵐商会,選別,460,460,kg,65.0,"29,900",渡辺,7996,ｼﾞｪｲｼﾃｨｰ,1,処分費,30
7,2025-03-31,ﾋﾗｶﾜ,GC 軽鉄･ｽﾁｰﾙ類,1470,"1,470",kg,0.0,0,川部,428,ﾋﾗｶﾜ,3,有価物,68
8,2025-03-31,ﾅｲﾃｯｸｽ,選別,1220,"1,220",kg,70.0,"85,400",山本,6509,ﾅｲﾃｯｸｽ,1,処分費,30
9,2025-03-31,ﾃﾙｳｪﾙ東日本,選別(ｷｬﾋﾞﾈｯﾄ),650,650,kg,65.0,"42,250",渡辺,4418,ﾃﾙｳｪﾙ東日本,1,処分費,171


### 有価物_出荷

In [58]:
# def read_info():
    # --- ① マスターCSVの読み込み ---
config = get_template_config()["balance_sheet"]
master_path = config["master_csv_path"]["yuka_shipping"]
master_csv = load_master_and_template(master_path)

# --- ② 金額の更新 ---
key_cols = ["業者名"]

master_with_money = summary_apply(
    master_csv,
    df_shipping,
    key_cols,
    source_col="金額",
    target_col="値",  # target_colとsource_colが同じ
)

master_with_money

2025-05-08 11:01:20,514 [INFO] (summary_tools.py:80) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['業者名'], 集計列=金額 ➡ 書き込み列=値


,業者名,値
0,ｴｺﾏｯｸｽ,NaN
1,宮崎,"3,500570"
2,Y-maxx,NaN
3,ﾐﾀﾞｯｸ,NaN
4,東京紙業城東事業所,NaN
5,新洋,NaN
6,荒井商店【二次】,NaN


### 有価ヤード

In [69]:
from utils.config_loader import get_unit_price_table_csv
from logic.manage.utils.multiply_tools import multiply_columns

# def read_info():
    # --- ① マスターCSVの読み込み ---
config = get_template_config()["balance_sheet"]
master_path = config["master_csv_path"]["yuka_yard"]
master_csv = load_master_and_template(master_path)
master_csv

,品名,値
0,GAH鋼･鉄筋等,NaN
1,GC軽鉄･ｽﾁｰﾙ類,NaN
2,GC軽鉄・ｽﾁｰﾙ類,NaN
3,GD,NaN
4,ｱﾙﾐ類,NaN
5,ｽﾃﾝﾚｽ,NaN
6,ﾄﾗﾝｽ,NaN
7,ﾓｰﾀｰ,NaN
8,ﾗｼﾞｴﾀｰ,NaN
9,室外機,NaN


In [70]:

# --- ② 重量を ---
master_with_money = summary_apply(
    master_csv,
    df_shipping,
    key_cols=["品名"],
    source_col="数量",
    target_col="数量",  # target_colとsource_colが同じ
)

master_with_money

2025-05-08 11:03:33,326 [INFO] (summary_tools.py:80) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['品名'], 集計列=数量 ➡ 書き込み列=数量


,品名,値,数量
0,GAH鋼･鉄筋等,NaN,NaN
1,GC軽鉄･ｽﾁｰﾙ類,NaN,NaN
2,GC軽鉄・ｽﾁｰﾙ類,NaN,560
3,GD,NaN,NaN
4,ｱﾙﾐ類,NaN,450
5,ｽﾃﾝﾚｽ,NaN,NaN
6,ﾄﾗﾝｽ,NaN,NaN
7,ﾓｰﾀｰ,NaN,530
8,ﾗｼﾞｴﾀｰ,NaN,100
9,室外機,NaN,NaN


In [71]:
# 単価計算
unit_price_csv = get_unit_price_table_csv()
unit_price_csv=unit_price_csv[unit_price_csv["必要項目"]=="有価物"]

master_with_money2 = summary_apply(
    master_with_money,
    unit_price_csv,
    key_cols=["品名"],
    source_col="設定単価",
    target_col="設定単価",
)
master_with_money2


2025-05-08 11:03:33,793 [INFO] (summary_tools.py:80) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['品名'], 集計列=設定単価 ➡ 書き込み列=設定単価


,品名,値,数量,設定単価
0,GAH鋼･鉄筋等,NaN,NaN,31.5
1,GC軽鉄･ｽﾁｰﾙ類,NaN,NaN,31.5
2,GC軽鉄・ｽﾁｰﾙ類,NaN,560,31.5
3,GD,NaN,NaN,31.5
4,ｱﾙﾐ類,NaN,450,180.0
5,ｽﾃﾝﾚｽ,NaN,NaN,180.0
6,ﾄﾗﾝｽ,NaN,NaN,33.0
7,ﾓｰﾀｰ,NaN,530,60.0
8,ﾗｼﾞｴﾀｰ,NaN,100,31.5
9,室外機,NaN,NaN,28.0


In [72]:
# --- ④ 単価×正味重量の計算 ---
result_df = multiply_columns(
    master_with_money2, col1="設定単価", col2="数量", result_col="値"
    )

In [73]:
result_df

,品名,値,数量,設定単価
0,GAH鋼･鉄筋等,NaN,NaN,31.5
1,GC軽鉄･ｽﾁｰﾙ類,NaN,NaN,31.5
2,GC軽鉄・ｽﾁｰﾙ類,17640.0,560.0,31.5
3,GD,NaN,NaN,31.5
4,ｱﾙﾐ類,81000.0,450.0,180.0
5,ｽﾃﾝﾚｽ,NaN,NaN,180.0
6,ﾄﾗﾝｽ,NaN,NaN,33.0
7,ﾓｰﾀｰ,31800.0,530.0,60.0
8,ﾗｼﾞｴﾀｰ,3150.0,100.0,31.5
9,室外機,NaN,NaN,28.0


In [74]:
result_df = result_df.rename(columns={"品名": "大項目"})
result_df

,大項目,値,数量,設定単価
0,GAH鋼･鉄筋等,NaN,NaN,31.5
1,GC軽鉄･ｽﾁｰﾙ類,NaN,NaN,31.5
2,GC軽鉄・ｽﾁｰﾙ類,17640.0,560.0,31.5
3,GD,NaN,NaN,31.5
4,ｱﾙﾐ類,81000.0,450.0,180.0
5,ｽﾃﾝﾚｽ,NaN,NaN,180.0
6,ﾄﾗﾝｽ,NaN,NaN,33.0
7,ﾓｰﾀｰ,31800.0,530.0,60.0
8,ﾗｼﾞｴﾀｰ,3150.0,100.0,31.5
9,室外機,NaN,NaN,28.0


## 売上

### 搬入台数

In [38]:
inbound_truck_count = df_receive["受入番号"].nunique()
inbound_truck_count

119

### 搬入量

In [39]:
inbound_weight = int(df_receive["正味重量"].sum())
inbound_weight

84180

### オネスト/kg　and オネストm3

In [40]:
from utils.config_loader import get_unit_price_table_csv
from logic.manage.utils.multiply_tools import multiply_columns

# def read_info():
    # --- ① マスターCSVの読み込み ---
config = get_template_config()["balance_sheet"]
master_path = config["master_csv_path"]["uriage"]
master_csv = load_master_and_template(master_path)
master_csv

,項目,伝票区分名,単位名,値
0,オネストkg,売上,NaN,NaN
1,オネストm3,売上,m3,NaN


In [41]:
# --- ② 重量を ---
master_m3 = summary_apply(
    master_csv,
    df_receive,
    key_cols=["伝票区分名","単位名"],
    source_col="金額",
    target_col="金額",  # target_colとsource_colが同じ
)

master_m3

2025-05-08 13:23:06,400 [INFO] (summary_tools.py:80) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['伝票区分名', '単位名'], 集計列=金額 ➡ 書き込み列=金額


,項目,伝票区分名,単位名,値,金額
0,オネストm3,売上,m3,NaN,294000.0
1,オネストkg,売上,NaN,NaN,NaN


In [42]:
# --- ② 重量を ---
master_kg = summary_apply(
    master_csv,
    df_receive,
    key_cols=["伝票区分名"],
    source_col="金額",
    target_col="金額",  # target_colとsource_colが同じ
)

master_kg

2025-05-08 13:23:06,440 [INFO] (summary_tools.py:80) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['伝票区分名'], 集計列=金額 ➡ 書き込み列=金額


,項目,伝票区分名,単位名,値,金額
0,オネストkg,売上,NaN,NaN,5029470.0
1,オネストm3,売上,m3,NaN,5029470.0


In [43]:
# オネストm3 金額を設定
value_m3 = master_m3.loc[master_m3["項目"] == "オネストm3", "金額"].values[0]
master_csv.loc[master_csv["項目"] == "オネストm3", "値"] = value_m3

# オネストkg 金額を設定（kg = 合計 - m3）
value_kg = master_kg.loc[master_kg["項目"] == "オネストkg", "金額"].values[0]
master_csv.loc[master_csv["項目"] == "オネストkg", "値"] = value_kg - value_m3


In [44]:
master_csv

,項目,伝票区分名,単位名,値
0,オネストkg,売上,NaN,4735470.0
1,オネストm3,売上,m3,294000.0


### 有価買取

In [45]:
# def read_info():
    # --- ① マスターCSVの読み込み ---
config = get_template_config()["balance_sheet"]
master_path = config["master_csv_path"]["uriage_yuka_kaitori"]
master_csv = load_master_and_template(master_path)
master_csv

,伝票区分名,単位名,品名,単価,値
0,支払,kg,NaN,NaN,NaN
1,NaN,NaN,ﾗｼﾞｴﾀｰ,0.0,NaN
2,NaN,NaN,ｷﾞﾛ材,0.0,NaN
3,NaN,NaN,銅,0.0,NaN
4,NaN,NaN,ﾄﾗﾝｽ,0.0,NaN
5,NaN,NaN,給湯器,0.0,NaN
6,NaN,NaN,鉄千地,0.0,NaN
7,NaN,NaN,配線,0.0,NaN
8,NaN,NaN,ｱﾙﾐ類,0.0,NaN
9,NaN,NaN,ｽﾃﾝﾚｽ,0.0,NaN


In [46]:
df_receive.columns.tolist()

['受入番号', '正味重量', '伝票区分名', '単位名', '単価', '金額', '品名', '数量']

In [47]:
#品名＆単価

# --- ② 重量 ---
master_csv2 = summary_apply(
    master_csv,
    df_receive,
    key_cols=["品名","単価"],
    source_col="数量",
    target_col="数量",  # target_colとsource_colが同じ
)

master_csv2


2025-05-08 13:23:06,541 [INFO] (summary_tools.py:80) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['品名', '単価'], 集計列=数量 ➡ 書き込み列=数量


,伝票区分名,単位名,品名,単価,値,数量
0,NaN,NaN,ﾗｼﾞｴﾀｰ,0.0,NaN,NaN
1,NaN,NaN,ｷﾞﾛ材,0.0,NaN,NaN
2,NaN,NaN,銅,0.0,NaN,NaN
3,NaN,NaN,ﾄﾗﾝｽ,0.0,NaN,NaN
4,NaN,NaN,給湯器,0.0,NaN,NaN
5,NaN,NaN,鉄千地,0.0,NaN,530.0
6,NaN,NaN,配線,0.0,NaN,NaN
7,NaN,NaN,ｱﾙﾐ類,0.0,NaN,NaN
8,NaN,NaN,ｽﾃﾝﾚｽ,0.0,NaN,NaN
9,NaN,NaN,GA H鋼･鉄筋等,0.0,NaN,NaN


In [48]:

# 単価計算
unit_price_csv = get_unit_price_table_csv()
unit_price_csv=unit_price_csv[unit_price_csv["必要項目"]=="有価買取"]

master_csv3 = summary_apply(
    master_csv2,
    unit_price_csv,
    key_cols=["品名","単価"],
    source_col="設定単価",
    target_col="設定単価",
)
master_csv3


2025-05-08 13:23:06,587 [INFO] (summary_tools.py:80) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['品名', '単価'], 集計列=設定単価 ➡ 書き込み列=設定単価


,伝票区分名,単位名,品名,単価,値,数量,設定単価
0,NaN,NaN,ﾗｼﾞｴﾀｰ,0.0,NaN,NaN,50.0
1,NaN,NaN,ｷﾞﾛ材,0.0,NaN,NaN,15.0
2,NaN,NaN,銅,0.0,NaN,NaN,800.0
3,NaN,NaN,ﾄﾗﾝｽ,0.0,NaN,NaN,15.0
4,NaN,NaN,給湯器,0.0,NaN,NaN,50.0
5,NaN,NaN,鉄千地,0.0,NaN,530.0,5.0
6,NaN,NaN,配線,0.0,NaN,NaN,50.0
7,NaN,NaN,ｱﾙﾐ類,0.0,NaN,NaN,50.0
8,NaN,NaN,ｽﾃﾝﾚｽ,0.0,NaN,NaN,50.0
9,NaN,NaN,GA H鋼･鉄筋等,0.0,NaN,NaN,15.0


In [49]:
# --- ④ 単価×正味重量の計算 ---
result_df = multiply_columns(
    master_csv3, col1="設定単価", col2="数量", result_col="値"
    )
result_df

,伝票区分名,単位名,品名,単価,値,数量,設定単価
0,NaN,NaN,ﾗｼﾞｴﾀｰ,0.0,NaN,NaN,50.0
1,NaN,NaN,ｷﾞﾛ材,0.0,NaN,NaN,15.0
2,NaN,NaN,銅,0.0,NaN,NaN,800.0
3,NaN,NaN,ﾄﾗﾝｽ,0.0,NaN,NaN,15.0
4,NaN,NaN,給湯器,0.0,NaN,NaN,50.0
5,NaN,NaN,鉄千地,0.0,2650.0,530.0,5.0
6,NaN,NaN,配線,0.0,NaN,NaN,50.0
7,NaN,NaN,ｱﾙﾐ類,0.0,NaN,NaN,50.0
8,NaN,NaN,ｽﾃﾝﾚｽ,0.0,NaN,NaN,50.0
9,NaN,NaN,GA H鋼･鉄筋等,0.0,NaN,NaN,15.0


In [55]:
# 支払kg
master_csv4 = summary_apply(
    master_csv3,
    df_receive,
    key_cols=["伝票区分名"],
    source_col="金額",
    target_col="値",  # target_colとsource_colが同じ
)

value_kg = master_csv4.loc[master_csv4["伝票区分名"] == "支払", "値"].values[0]
result_df.loc[result_df["伝票区分名"] == "支払", "値"] = value_kg

int(result_df["値"].sum())

2025-05-08 13:28:52,186 [INFO] (summary_tools.py:80) [dd3b4088c738/root] ▶️ マスター更新処理: キー=['伝票区分名'], 集計列=金額 ➡ 書き込み列=値


98115